Author: Rudra Panda

Q1) Load the dataset into a pandas dataframe and display the first 5 lines of the dataset along with the column headings.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#loading the dataset
dataset = pd.read_table('data.txt', delim_whitespace=True, 
                   names=('CRIM', 'ZN', 'INDUS', 'CHAS', 
                          'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
                         'B', 'LSTAT', 'MEDV'))
#display the first 5 lines of the dataset along with the column headings
dataset.head(n=5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Q2) Split the dataset into training (70%) and testing set (30%)

In [2]:
#features and targt
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 13].values

#add intercept
import statsmodels.formula.api as sm
X = np.append(arr=np.ones((len(X),1)).astype(int), values=X, axis=1)

#Splitting data into Training and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

#Normalizing the features
#from sklearn.preprocessing import StandardScaler
#sc_X = StandardScaler()
#X_train = sc_X.fit_transform(X_train)
#X_test = sc_X.transform(X_test)

Q3) Model 1: Build a linear regression model with all the variables using Normal Equations.

In [3]:
#for inverse matrix
from numpy.linalg import inv

#apply normal equation
w = inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)

Q4) What are the weight parameters you get for Model 1.

In [4]:
print(w)

[ 3.79371077e+01 -1.21310401e-01  4.44664254e-02  1.13416945e-02
  2.51124642e+00 -1.62312529e+01  3.85906801e+00 -9.98516565e-03
 -1.50026956e+00  2.42143466e-01 -1.10716124e-02 -1.01775264e+00
  6.81446545e-03 -4.86738066e-01]


Q5) Using Model 1 make a prediction on the test set. Calculate mean squared error.

In [5]:
# Make a prediction on test
y_pred_ne = X_test.dot(w.T)

# Calculate mean squared error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred_ne)

27.195965766882875

Q6) Model 2: Using scikit learn, build a Linear Regression model with all the variables.

In [6]:
from sklearn.linear_model import LinearRegression

#features and targt for LR
X_LR = dataset.iloc[:, :-1].values
y_lr = dataset.iloc[:, 13].values

#Splitting data into Training and Test Set
from sklearn.model_selection import train_test_split
X_LR_train, X_LR_test, y_lr_train, y_lr_test = train_test_split(X_LR,y_lr,test_size=0.3,random_state=0)

lrObj = LinearRegression()
lrObj.fit(X_LR_train, y_lr_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Q7) What are the weight parameters you get for Model 2.

In [7]:
print(lrObj.coef_)

[-1.21310401e-01  4.44664254e-02  1.13416945e-02  2.51124642e+00
 -1.62312529e+01  3.85906801e+00 -9.98516565e-03 -1.50026956e+00
  2.42143466e-01 -1.10716124e-02 -1.01775264e+00  6.81446545e-03
 -4.86738066e-01]


Q8) Using Model 2 make a prediction on the test set. Calculate mean squared error.

In [8]:
y_pred_lr = lrObj.predict(X_LR_test)

# Calculate mean squared error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_lr_test, y_pred_lr)

27.195965766883425

Q9) Write a function forwardSelection(x, sl) to implement the forward feature selection technique.

In [9]:
import statsmodels.formula.api as sm

def forwardSelection(x, sl):
    input_features = list(range(0, len(x[0])))
    selected_features = []
    
    while (len(input_features) > 0):
        minimum = None
        for feature in input_features:
            trial_combination_features = selected_features + [feature]
            olsObj = sm.OLS(y,x[:,trial_combination_features]).fit()
            varPosition = trial_combination_features.index(feature)
            featurePvalue = (olsObj.pvalues[varPosition]).astype(float)
            if minimum == None or featurePvalue < minimum:
                minimum = featurePvalue
                idx = trial_combination_features
        
        if minimum < sl:
            selected_features = idx
            selected_features.sort()
            input_features = [i for i in input_features if i not in selected_features]
        else:
            break
    
    x = x[:,selected_features]
    print(selected_features)
    return x

Q10) Display the features selected by your function

In [10]:
sig_level = 0.05
X_sig = forwardSelection(X, sig_level)

[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13]


Q11) Model 3: Using sckikit learn, build a Linear Regression model using only the features selected by the forward selection function you implemented.

In [11]:
#Splitting the data into Training Set and Test Set
X_sig_train, X_sig_test, y_sig_train, y_sig_test= train_test_split(X_sig, y, test_size=0.3,random_state=0)
mlrObj_sig= LinearRegression()
mlrObj_sig.fit(X_sig_train, y_sig_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Q12) Using Model 3 make a prediction on the test set. Calculate mean squared error.

In [12]:
y_sig_pred= mlrObj_sig.predict(X_sig_test)
mean_squared_error(y_sig_test, y_sig_pred)

27.046570095927265

In [13]:
def prof_forwardSelection(x, sl):
    row, column = x.shape
    minIndex = -2
    k = column

    #loop through k-1 size combinations starting at 0 (i)
    for i in range(0, k+1):
        #print("i: " + str(i) + " | X" + str(minIndex) + " added to model" +  " | " + str(x[0]))
        if i > 1 and minIndex != -1:
            selected = np.column_stack((selected[:,], x[:,minIndex]))
            x = np.delete(x, minIndex, 1)
        elif i == 1 and minIndex != -1:
            selected = x[:,minIndex]
            x = np.delete(x, minIndex, 1)
        
        minPval = 1
        minIndex = -1
        #loop through columns k-i times (j) starting at 0
        for j in range(0, k-i):
            if i > 0:
                obj_OLS = sm.OLS(y, np.column_stack((selected[:,], x[:,j]))).fit()
            else:
                obj_OLS = sm.OLS(y, x[:,j]).fit()
 
            pVal = obj_OLS.pvalues[-1].astype(float)
            print(obj_OLS.pvalues)
            #print("----X" + str(j) + ": p-value = " + str(pVal) + " | Min P-Value = " + str(minPval) + "-----")
            if pVal < sl:
                if pVal < minPval:
                    minPval = pVal
                    minIndex = j
    return selected

In [14]:
sig_level = 0.05
X_sig = forwardSelection(X, sig_level)

[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13]
